## checking global property of zero sums

In [1]:
s0 = [0, 6,14, 1,15, 4, 7,13, 9, 8,12, 5, 2,10, 3,11]
s1 = [0, 9,13, 2,15, 1,11, 7, 6, 4, 5, 3, 8,12,10,14]

def check_permutation(s, name):
    if sorted(s) == list(range(16)):
        print(f"{name} is a permutation of 0..15")
    else:
        print(f"{name} is NOT a permutation")

def check_zerosum(s, name):
    xor_sum = 0
    for x in s:
        xor_sum ^= x
    print(f"{name} XOR sum: {xor_sum}")
    if xor_sum == 0:
        print(f"{name} satisfies zero-sum property (sum is 0)")
    else:
        print(f"{name} does NOT satisfy zero-sum property")

print("Verifying S-boxes...")
check_permutation(s0, "s0")
check_zerosum(s0, "s0")
print("-" * 20)
check_permutation(s1, "s1")
check_zerosum(s1, "s1")

Verifying S-boxes...
s0 is a permutation of 0..15
s0 XOR sum: 0
s0 satisfies zero-sum property (sum is 0)
--------------------
s1 is a permutation of 0..15
s1 XOR sum: 0
s1 satisfies zero-sum property (sum is 0)


## finding any subset which is satisfying the zero sum property

In [4]:
from itertools import combinations

def find_nontrivial_zero_sum_subset(sbox, max_subset_size=None):
    n = len(sbox)
    if max_subset_size is None:
        max_subset_size = n  # try all sizes 2..n

    for r in range(2, max_subset_size + 1):  # skip size 1
        for idxs in combinations(range(n), r):
            acc = 0
            for i in idxs:
                acc ^= sbox[i]
            if acc == 0:
                return True, idxs
    return False, None

for name, sbox in {"S0": s0, "S1": s1}.items():
    found, subset = find_nontrivial_zero_sum_subset(sbox, 8)
    if found:
        print(f"{name}: Found nontrivial zero-sum subset {subset}")
        print(f"Inputs: {subset}")
        print(f"Outputs: {[hex(sbox[i]) for i in subset]}")
    else:
        print(f"{name}: No nontrivial zero-sum subset (up to size 8).")


S0: Found nontrivial zero-sum subset (1, 2, 9)
Inputs: (1, 2, 9)
Outputs: ['0x6', '0xe', '0x8']
S1: Found nontrivial zero-sum subset (1, 2, 9)
Inputs: (1, 2, 9)
Outputs: ['0x9', '0xd', '0x4']


## higher order differntial distinguishers

### step 1: finding algabraic degree of the s box

In [5]:
import itertools

S0 = [0, 6, 14, 1, 15, 4, 7, 13, 9, 8, 12, 5, 2, 10, 3, 11]
S1 = [0, 9, 13, 2, 15, 1, 11, 7, 6, 4, 5, 3, 8, 12, 10, 14]

def get_bit(x, bit):
    return (x >> bit) & 1

def derivative(f, direction, n):
    """Compute first-order derivative of Boolean function f in given direction."""
    return [f[x] ^ f[x ^ direction] for x in range(1 << n)]

def degree_of_boolean_function(f, n):
    """Find degree using iterative derivatives."""
    deg = 0
    derivatives = {tuple(f)}
    while True:
        new_derivatives = set()
        for d in range(1 << n):
            for val in derivatives:
                val_list = list(val)
                df = derivative(val_list, d, n)
                if any(df):  # nonzero derivative exists
                    new_derivatives.add(tuple(df))
        if not new_derivatives:  # all derivatives became zero
            return deg
        derivatives = new_derivatives
        deg += 1

def sbox_degree(sbox, n=4, m=4):
    degrees = []
    for bit in range(m):
        f = [get_bit(y, bit) for y in sbox]
        deg = degree_of_boolean_function(f, n)
        degrees.append(deg)
    return max(degrees), degrees

print("S0 degree:", sbox_degree(S0))
print("S1 degree:", sbox_degree(S1))


S0 degree: (3, [3, 3, 3, 3])
S1 degree: (3, [3, 3, 3, 3])


In [17]:
%run ../toy/toy.ipynb


Input state:      [1, 0, 0, 0, 0, 0, 0, 0]
After S-box:      [0, 1, 1, 0, 0, 0, 0, 0]
After S-box inv:  [1, 0, 0, 0, 0, 0, 0, 0]
Input State:      [1, 2, 3, 4, 5, 6, 7, 8]
After SR sheet:   [1, 2, 3, 4, 5, 9, 13, 2]
After inv SR:     [1, 2, 3, 4, 5, 6, 7, 8]
Original: [0, 0, 0, 1, 0, 0, 0, 0]
After MDS: [0, 1, 1, 0, 0, 1, 0, 0]
After inverse: [0, 0, 0, 1, 0, 0, 0, 0]
Correct: True
Average diffusion: 4.25 output nibbles changed per input bit flip
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
Decrypted ciphertext: [1, 0, 0, 0, 0, 0, 0, 0]
[13, 0, 3, 2, 6, 1, 2, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
sahi aya kya :  True


In [18]:
from itertools import product
import numpy as np

def test_higher_order_zero_sum(R=1, num_trials=5):
    """
    Verify the 4th-order zero-sum property for your toy Saturnin.
    Each test forms an affine cube of 16 plaintexts (4 active bits)
    and XORs all ciphertexts to see if they sum to zero.
    """
    active_bits = [0, 1, 2, 3]    # 4 active bits → 2^4 = 16 points
    key = [0]*8                   # fixed zero key

    print(f"\nTesting higher-order zero-sum property for {R} round(s):\n")

    for trial in range(num_trials):
        # random 8-byte base plaintext
        base = np.random.randint(0, 256, size=8, dtype=np.uint8)

        # build 16 plaintexts by toggling 4 bits of byte[0]
        cube_pts = []
        for selector in product([0,1], repeat=len(active_bits)):
            p = base.copy()
            for bit, s in zip(active_bits, selector):
                if s:
                    p[0] ^= (1 << bit)
            cube_pts.append(p)

        # encrypt all 16 plaintexts with toy Saturnin
        ciphertexts = [encrypt_toy_debug(pt.copy(), key.copy(), R=R)
                       for pt in cube_pts]

        # XOR all ciphertexts bytewise
        xor_sum = np.bitwise_xor.reduce(ciphertexts)

        print(f"Trial {trial+1}: XOR of 16 ciphertexts = {xor_sum}")
        if np.all(xor_sum == 0):
            print("  ✅ Zero-sum (4th-order derivative vanished)")
        else:
            print("  ❌ Non-zero (degree > 3 or diffusion increased)")

# Run the test for 1 round
test_higher_order_zero_sum(R=1, num_trials=5)



Testing higher-order zero-sum property for 1 round(s):

Trial 1: XOR of 16 ciphertexts = [0 0 0 0 0 0 0 0]
  ✅ Zero-sum (4th-order derivative vanished)
Trial 2: XOR of 16 ciphertexts = [0 0 0 0 0 0 0 0]
  ✅ Zero-sum (4th-order derivative vanished)
Trial 3: XOR of 16 ciphertexts = [0 0 0 0 0 0 0 0]
  ✅ Zero-sum (4th-order derivative vanished)
Trial 4: XOR of 16 ciphertexts = [0 0 0 0 0 0 0 0]
  ✅ Zero-sum (4th-order derivative vanished)
Trial 5: XOR of 16 ciphertexts = [0 0 0 0 0 0 0 0]
  ✅ Zero-sum (4th-order derivative vanished)


In [19]:
for r in range(1, 5):
    test_higher_order_zero_sum(R=r, num_trials=3)



Testing higher-order zero-sum property for 1 round(s):

Trial 1: XOR of 16 ciphertexts = [0 0 0 0 0 0 0 0]
  ✅ Zero-sum (4th-order derivative vanished)
Trial 2: XOR of 16 ciphertexts = [0 0 0 0 0 0 0 0]
  ✅ Zero-sum (4th-order derivative vanished)
Trial 3: XOR of 16 ciphertexts = [0 0 0 0 0 0 0 0]
  ✅ Zero-sum (4th-order derivative vanished)

Testing higher-order zero-sum property for 2 round(s):

Trial 1: XOR of 16 ciphertexts = [0 0 0 0 0 0 0 0]
  ✅ Zero-sum (4th-order derivative vanished)
Trial 2: XOR of 16 ciphertexts = [0 0 0 0 0 0 0 0]
  ✅ Zero-sum (4th-order derivative vanished)
Trial 3: XOR of 16 ciphertexts = [0 0 0 0 0 0 0 0]
  ✅ Zero-sum (4th-order derivative vanished)

Testing higher-order zero-sum property for 3 round(s):

Trial 1: XOR of 16 ciphertexts = [ 9  1  1  7  8  3  4 13]
  ❌ Non-zero (degree > 3 or diffusion increased)
Trial 2: XOR of 16 ciphertexts = [ 9  0 13  1  2 13  2 13]
  ❌ Non-zero (degree > 3 or diffusion increased)
Trial 3: XOR of 16 ciphertexts = [14 